In [401]:
import pandas as pd
from datetime import datetime
import numpy as np

In [402]:
df = pd.read_csv("GooglePlay.csv")
# df = df.drop_duplicates(['App', 'Category', "Rating", "Reviews", "Size", "Installs", "Type", "Price", "Content Rating", "Genres", "Last Updated", "Current Ver", "Android Ver", "Unnamed: 14"])
df.head(50)
# df.shape

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Unnamed: 14
0,0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up,NaN
1,1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design,Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up,NaN
3,3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up,NaN
4,4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design,Creativity,20-Jun-18,1.1,4.4 and up
5,5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,26-Mar-17,1,2.3 and up,NaN
6,6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,26-Apr-18,1.1,4.0.3 and up,NaN
7,7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,14-Jun-18,6.1.61.1,4.2 and up,NaN
8,8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,20-Sep-17,2.9.2,3.0 and up,NaN
9,9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design,Creativity,3-Jul-18,2.8,4.0.3 and up


In [403]:
output_cols = ['attribute name', "record count", 'null count', 'completeness']
output = []
for col in df.columns:
    record = []
    record.append(col)
    record.append(df[col].shape[0])
    record.append(df[df[col].isnull()].shape[0])
    completeness = 100.0 * float(df[col].shape[0] - df[df[col].isnull()].shape[0]) / float(df[col].shape[0])
    completeness = float("{:.2f}".format(completeness))
    record.append(completeness)
    output.append(record)
    # pass

output_df = pd.DataFrame(output, columns=output_cols)
output_df

,attribute name,record count,null count,completeness
0,Unnamed: 0,10841,0,100.00
1,App,10841,0,100.00
2,Category,10841,0,100.00
3,Rating,10841,1474,86.40
4,Reviews,10841,0,100.00
5,Size,10841,0,100.00
6,Installs,10841,0,100.00
7,Type,10841,1,99.99
8,Price,10841,0,100.00
9,Content Rating,10841,1,99.99


#### Currentness( the current is considered after the 2018-01-01)

In [404]:

s = r'\b(0?[1-9]|[12][0-9]|3[01])-(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)-(\d+)\b'
current_date = pd.to_datetime("2018-01-01").date()
currentness = pd.to_datetime(df.loc[df['Last Updated'].str.match(s), 'Last Updated'], format="%d-%b-%y").dt.date >= current_date
count_current = df.loc[df['Last Updated'].str.match(s), 'Last Updated'][currentness].shape[0]
total = df.shape[0]
for index, row in output_df.iterrows():
    output_df.at[index, "Currentness"] = float("{:.2f}".format(100.0 * float(count_current) / float(total)))
output_df

,attribute name,record count,null count,completeness,Currentness
0,Unnamed: 0,10841,0,100.00,64.61
1,App,10841,0,100.00,64.61
2,Category,10841,0,100.00,64.61
3,Rating,10841,1474,86.40,64.61
4,Reviews,10841,0,100.00,64.61
5,Size,10841,0,100.00,64.61
6,Installs,10841,0,100.00,64.61
7,Type,10841,1,99.99,64.61
8,Price,10841,0,100.00,64.61
9,Content Rating,10841,1,99.99,64.61


#### duplicate records detected based on App name

In [405]:
df[df['App'].isin(df["App"].value_counts()[df["App"].value_counts() >1].index)].sort_values('App')

,Unnamed: 0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Unnamed: 14
1393,1393,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3.8M,"500,000+",Free,0,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up,NaN
1407,1407,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3.8M,"500,000+",Free,0,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up,NaN
2543,2543,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26M,"1,000,000+",Free,0,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up,NaN
2322,2322,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26M,"1,000,000+",Free,0,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up,NaN
2385,2385,2017 EMRA Antibiotic Guide,MEDICAL,4.4,12,3.8M,"1,000+",Paid,$16.99,Everyone,Medical,27-Jan-17,1.0.5,4.0.3 and up,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3202,3202,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,4.2,219848,Varies with device,"50,000,000+",Free,0,Everyone,Travel & Local,2-Aug-18,Varies with device,Varies with device,NaN
3118,3118,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,4.2,219848,Varies with device,"50,000,000+",Free,0,Everyone,Travel & Local,2-Aug-18,Varies with device,Varies with device,NaN
3103,3103,trivago: Hotels & Travel,TRAVEL_AND_LOCAL,4.2,219848,Varies with device,"50,000,000+",Free,0,Everyone,Travel & Local,2-Aug-18,Varies with device,Varies with device,NaN
8291,8291,wetter.com - Weather and Radar,WEATHER,4.2,189310,38M,"10,000,000+",Free,0,Everyone,Weather,6-Aug-18,Varies with device,Varies with device,NaN


### Validity

In [406]:
category_invalid = 0
category_invalid += df['Category'].str.contains('\d', regex=True).sum()
category_invalid +=  output_df.loc[output_df['attribute name'] == "Category", "null count"]
category_valid = df.shape[0] - category_invalid
validity = 100.0 * category_valid / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Category", "validity"] = float("{:.2f}".format(validity.values[0]))


In [407]:
rating_invalid = 0
rating_invalid += df[df['Rating'] > 5].shape[0]
rating_invalid += output_df.loc[output_df['attribute name'] == "Rating", "null count"].values[0]
rating_valid = df.shape[0] - rating_invalid
validity = 100.0 * float(rating_valid) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Rating", "validity"] = float("{:.2f}".format(validity))


In [408]:
output_df.loc[output_df['attribute name'] == "Unnamed: 0", "validity"] = float(100.0)
output_df.loc[output_df['attribute name'] == "App", "validity"] = float(100.0)
# output_df

In [409]:
# df[df['Unnamed: 14'].notnull()]['Current Ver']
s = r'^\d+(\.\d+)+$'
t = df[df['Current Ver'].notnull()]
valid_current_ver = t[t['Current Ver'].str.match(s)].shape[0]
validity = 100.0 * float(valid_current_ver) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Current Ver", "validity"] = float(validity)
# output_df

In [410]:
s = r'^\d+(\.\d+)?(?:\sand\s+up)?$'
t = df[df['Android Ver'].notnull()]
valid_current_ver = t[t['Android Ver'].str.match(s)].shape[0]
validity = 100.0 * float(valid_current_ver) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Android Ver", "validity"] = float(validity)
# output_df

In [411]:
s = r'\b(0?[1-9]|[12][0-9]|3[01])-(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)-(\d+)\b'
t = df[df['Last Updated'].notnull()]
valid_last_update = t[t['Last Updated'].str.match(s)].shape[0]
validity = 100.0 * float(valid_last_update) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Last Updated", "validity"] = float(validity)
# output_df

In [412]:
t = df[df['Genres'].notnull()]
valid_genre = t[t['Genres'].str.match(r"[a-zA-Z]+(\s&\s[a-zA-Z]+)*")].shape[0]
validity = 100.0 * float(valid_genre) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Genres", "validity"] = float(validity)
# output_df

In [413]:
valid_reviews = df[df['Reviews'].str.match('\d')].shape[0]
validity = 100.0 * float(valid_reviews) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Reviews", "validity"] = float(validity)
# output_df

In [414]:
valid_size = df[df['Size'].str.match(r"\d+(\.\d)?[Mk]")].shape[0]
validity = 100.0 * float(valid_size) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Size", "validity"] = float(validity)
# output_df

In [415]:
s = r'\d{1,3}(,\d{3})*\+?'
valid_installs = df[df['Installs'].str.match(s)].shape[0]
validity = 100.0 * float(valid_installs) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Installs", "validity"] = float(validity)
# output_df

In [416]:
s = r'\b(Free|Paid)\b'
t = df[df['Type'].notnull()]
valid_type = t[t['Type'].str.match(s)].shape[0]
validity = 100.0 * float(valid_type) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Type", "validity"] = float(validity)
# output_df

In [417]:
s = r'\$?\d+(\.\d)?'
valid_price = df[df['Price'].str.match(s)].shape[0]
validity = 100.0 * float(valid_price) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Price", "validity"] = float(validity)
# output_df

In [421]:
s = r'\b(Unrated)\b'
t = df[df['Content Rating'].notnull()]
valid_content_rating = t[~t["Content Rating"].str.match(s)].shape[0]
validity = 100.0 * float(valid_content_rating) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Content Rating", "validity"] = float(validity)


### consistency

In [450]:
output_df.loc[output_df['attribute name'] == "Unnamed: 0", "consistency"] = float(100)
output_df.loc[output_df['attribute name'] == "App", "consistency"] = float(100)

output_df

,attribute name,record count,null count,completeness,Currentness,validity,consistency
0,Unnamed: 0,10841,0,100.00,64.61,100.000000,100.000000
1,App,10841,0,100.00,64.61,100.000000,100.000000
2,Category,10841,0,100.00,64.61,99.990000,99.990776
3,Rating,10841,1474,86.40,64.61,86.390000,NaN
4,Reviews,10841,0,100.00,64.61,100.000000,NaN
5,Size,10841,0,100.00,64.61,84.355687,NaN
6,Installs,10841,0,100.00,64.61,99.990776,NaN
7,Type,10841,1,99.99,64.61,99.981552,NaN
8,Price,10841,0,100.00,64.61,99.990776,NaN
9,Content Rating,10841,1,99.99,64.61,99.972327,NaN


In [449]:
count = 0
for i, r in df.iterrows():
    t = r['Genres'].replace("&", "AND").replace(" ", "_").upper()
    c = r['Category']
    if not c in t and c != 'GAME' and c != 'FAMILY':
        count += 1

consistency = 100.0 * float(df.shape[0] - count) / float(df.shape[0])
output_df.loc[output_df['attribute name'] == "Genres", "consistency"] = consistency
output_df.loc[output_df['attribute name'] == "Category", "consistency"] = consistency
output_df

,attribute name,record count,null count,completeness,Currentness,validity,consistency
0,Unnamed: 0,10841,0,100.00,64.61,100.000000,100.000000
1,App,10841,0,100.00,64.61,100.000000,NaN
2,Category,10841,0,100.00,64.61,99.990000,99.990776
3,Rating,10841,1474,86.40,64.61,86.390000,NaN
4,Reviews,10841,0,100.00,64.61,100.000000,NaN
5,Size,10841,0,100.00,64.61,84.355687,NaN
6,Installs,10841,0,100.00,64.61,99.990776,NaN
7,Type,10841,1,99.99,64.61,99.981552,NaN
8,Price,10841,0,100.00,64.61,99.990776,NaN
9,Content Rating,10841,1,99.99,64.61,99.972327,NaN
